# Exploratory Database Querying in SQL

In this project, I use SQL to run some simple queries on a sample database provided by Microsoft: The AdventureWorks Sample Database. The database is a collection of simple  tables of synthetic data about a bike parts manufacturer arranged with a simple schema. It can be hosted on a cloud server and accessed with an online database driver to practice SQL queries which is what I do in this project.

Let's have a look at this data!

## Implementation Background Info

This is a self-guided project that I planned, initiated, and performed original analysis for purely as a personal interest project. When looking for simple ways to practice SQL, I learned that Microsoft allows Azure users a [free trial of its Azure SQL Database service](https://learn.microsoft.com/en-us/azure/azure-sql/database/free-offer?view=azuresql) in which they can host a few databases for free. Not only do they provide this, they also have a sample database that users can host on their servers to test both database querying and application production among other things. I created a server on which I host a copy of this free database and connected to it remotely using the `pyodbc` package as shown in the first few code cells below:

In [13]:
### Packages

import pyodbc
import os
import pandas as pd

In [14]:
### Connecting to the Database
# Connection string
server = "canyen-server-1.database.windows.net"
database = "test-db-1"
username = os.environ['username']
password = os.environ['password']
driver = "{/usr/local/lib/libmsodbcsql.18.dylib}"

connection_string = f"Driver={driver};\
                    Server=tcp:{server},1433;\
                    Database={database};\
                    Uid={username};\
                    Pwd={password};\
                    Encrypt=yes;\
                    TrustServerCertificate=no;\
                    Connection Timeout=120;"

# Make connection
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful")
except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"Connection error: {sqlstate}")

Connection successful


The "Connection successful" message means I am now connected to my Azure server in my Notebook session.

*Note that in the code above, the username and password strings are not hard-coded but were first assigned to the environment using code run only in the console and then referenced for security.*

In [15]:
# Convenience Function
def query_result(query):
    # Execute
    cursor = conn.cursor()
    cursor.execute(query)
    
    # Format results
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]

    result = pd.DataFrame.from_records(rows, columns = columns)
    return(result)

## The AdventureWorks Database

As stated above, the AdventureWorks Database is a database of synthetic data about a bike parts manufacturer. The database has tables about all kinds of aspects of this hypothetical company's business operations, including sales records, product information, and customer lists among other things. From the context of the analysis results, AdventureWorks appears to make most of its sales to other businesses who then stock and sell AdventureWorks' products.

Here are the names of all the tables:

In [16]:
### Viewing Tables in the Database
# Names of tables
query = """

SELECT name 
  FROM sys.tables;
  
"""

query_result(query)

,name
0,Customer
1,ProductModel
2,ProductDescription
3,Product
4,ProductModelProductDescription
5,ProductCategory
6,BuildVersion
7,ErrorLog
8,Address
9,CustomerAddress


*Note also that the particular version of SQL supported by Azure SQL Database is "Microsoft SQL Server" which has a number of differences in syntax/function support than other versions such as SQLite and Oracle SQL.*

The "name" field from "sys.tables" actually only shows the informal table names as might be shown on a schema. In reality, all the data tables have the prefix "SalesLT." in their names which need to be used in the query.

Here are all the columns in the tables named "SalesLT.SalesOrderDetail" table:

In [17]:
# Columns of Sales Detail Table
query = """

SELECT name
  FROM sys.columns
 WHERE object_id = OBJECT_ID('SalesLT.SalesOrderDetail');
  
"""
   
query_result(query)

,name
0,SalesOrderID
1,SalesOrderDetailID
2,OrderQty
3,ProductID
4,UnitPrice
5,UnitPriceDiscount
6,LineTotal
7,rowguid
8,ModifiedDate


The "SalesOrderDetail" table is a condensed table of sales information, with the "SalesOrderHeader" table having a more extensive collection of different columns related to sales. In the Azure SQL Database online platform, it also shows that the "SalesOrderID" and "SalesOrderDetailID" columns are both primary keys and together make the full key.

## Analysis of Revenue

Now that I have familiarized myself with the database, I'll do some analysis of **revenue** represented in the database. This is analysis that I strongly suspect would have business implications for AdventureWorks were it a real company. Specifically, I am interested in the following:
* Which products bring in the most revenue?
* Which product categories bring in the most revenue?
* Which customers have spent the most?

In [18]:
### Which product has had the most sales of all time?
# How many orders are in the table?
query = """

SELECT COUNT(DISTINCT SalesOrderID)
  FROM SalesLT.SalesOrderDetail;
  
"""
   
query_result(query)

,
0,32


There are actually a few hundred rows in this table, but they are not unique. That is becuase this table is compiled by both order ID and product with multiple product sales typically being included in one order and one order ID.

In this query, I join the "SalesOrderDetail" table to the "Product" table to see which products have the most sales:

In [19]:
# Which products have had the most revenue?
query = """

SELECT TOP 10 P.Name, S.ProductID, SUM(S.LineTotal) AS TotalRevenue
  FROM SalesLT.SalesOrderDetail AS S
  LEFT JOIN SalesLT.Product AS P
    ON S.ProductID = P.ProductID
  GROUP BY P.Name, S.ProductID
  ORDER BY TotalRevenue DESC;
  
"""
   
query_result(query)

,Name,ProductID,TotalRevenue
0,"Touring-1000 Blue, 60",969,37191.492000
1,"Mountain-200 Black, 42",783,37178.838000
2,"Road-350-W Yellow, 48",976,36486.235500
3,"Mountain-200 Black, 38",782,35801.844000
4,"Touring-1000 Yellow, 60",957,23413.474656
5,"Touring-1000 Blue, 50",967,22887.072000
6,"Mountain-200 Silver, 42",780,20879.910000
7,"Road-350-W Yellow, 40",973,20411.880000
8,"Mountain-200 Black, 46",784,19277.916000
9,"Road-350-W Yellow, 42",974,18692.519308


There are three bike types that dominate in terms of total revenue in our database: they are the **Touring-1000**, the **Mountain-200**, and the **Road-350** -- each popular in a variety of colors and sizes.

The same query can be used to find the products that bring in the *least* revenue: the only necessary change is removing the `DESC` clause at the end of the query

In [20]:
### Which products have had the least revenue?
query = """

SELECT TOP 10 P.Name, S.ProductID, SUM(S.LineTotal) AS TotalRevenue
  FROM SalesLT.SalesOrderDetail AS S
  LEFT JOIN SalesLT.Product AS P
    ON S.ProductID = P.ProductID
  GROUP BY P.Name, S.ProductID
  ORDER BY TotalRevenue;
  
"""
   
query_result(query)

,Name,ProductID,TotalRevenue
0,Patch Kit/8 Patches,873,27.480000
1,HL Touring Handlebars,947,54.942000
2,HL Road Seat/Saddle,913,63.168000
3,Rear Brakes,907,63.900000
4,"Racing Socks, M",874,80.910000
5,Chain,952,97.152000
6,LL Mountain Seat/Saddle,908,97.632000
7,Water Bottle - 30 oz.,870,158.206952
8,"Half-Finger Gloves, S",858,176.328000
9,LL Mountain Handlebars,808,187.068000


So, while this company sells a wide array of bike goods, it makes far more in revenue in its actual bikes than it does in biking accessories.

However, revenue may not perfectly track with units sold of a particular product. Here are the top products by units sold:

In [21]:
# Which products have had the most units sold?
query = """

SELECT TOP 10 P.Name, S.ProductID, SUM(OrderQty) AS UnitsSold
  FROM SalesLT.SalesOrderDetail AS S
  LEFT JOIN SalesLT.Product AS P
    ON S.ProductID = P.ProductID
  GROUP BY P.Name, S.ProductID
  ORDER BY UnitsSold DESC;
  
"""
   
query_result(query)

,Name,ProductID,UnitsSold
0,"Classic Vest, S",864,87
1,"Short-Sleeve Classic Jersey, XL",884,57
2,Bike Wash - Dissolver,877,55
3,Water Bottle - 30 oz.,870,54
4,AWC Logo Cap,712,52
5,"Long-Sleeve Logo Jersey, L",715,51
6,"Sport-100 Helmet, Black",708,51
7,"Racing Socks, L",875,51
8,"Short-Sleeve Classic Jersey, L",883,51
9,Hydration Pack - 70 oz.,880,50


So, of all of AdventureWorks' products, their clothing is most represented among products of which they sell the most.

Which products had the most revenue when grouped by category? We know to expect bike categories -- especially touring, road, and mountain bikes -- to dominate, but by what factor(s)?

In [22]:
### Which categories have had the most revenue?
query = """

SELECT TOP 10 C.Name, C.ProductCategoryID, SUM(S.LineTotal) AS TotalRevenue
  FROM SalesLT.SalesOrderDetail AS S
  LEFT JOIN SalesLT.Product AS P
    ON S.ProductID = P.ProductID
  LEFT JOIN SalesLT.ProductCategory AS C
    ON P.ProductCategoryID = C.ProductCategoryID
  GROUP BY C.Name, C.ProductCategoryID
  ORDER BY TotalRevenue DESC;
  
"""
   
query_result(query)

,Name,ProductCategoryID,TotalRevenue
0,Touring Bikes,7,220655.375796
1,Road Bikes,6,183130.296808
2,Mountain Bikes,5,170825.886000
3,Mountain Frames,16,54949.602000
4,Road Frames,18,24346.584000
5,Touring Frames,20,19066.260000
6,Jerseys,25,7017.880276
7,Vests,29,4309.903750
8,Cranksets,12,3968.868000
9,Shorts,26,3299.804532


The full bikes side of AdventureWorks' business is the most lucrative by far. The company made nearly half a million dollars from its bike, whereas its next most popular product category, mountain frames, made only about $55,000.

And which categories brought in the least revenue?

In [23]:
### Which categories have had the least revenue?
query = """

SELECT TOP 10 C.Name, C.ProductCategoryID, SUM(S.LineTotal) AS TotalRevenue
  FROM SalesLT.SalesOrderDetail AS S
  LEFT JOIN SalesLT.Product AS P
    ON S.ProductID = P.ProductID
  LEFT JOIN SalesLT.ProductCategory AS C
    ON P.ProductCategoryID = C.ProductCategoryID
  GROUP BY C.Name, C.ProductCategoryID
  ORDER BY TotalRevenue;
  
"""
   
query_result(query)

,Name,ProductCategoryID,TotalRevenue
0,Tires and Tubes,41,27.480000
1,Chains,11,97.152000
2,Bottles and Cages,32,158.206952
3,Cleaners,33,251.875875
4,Caps,23,277.363076
5,Socks,27,345.208808
6,Brakes,10,830.700000
7,Gloves,24,837.558000
8,Saddles,19,1010.304000
9,Handlebars,8,1192.968000


These results track with other analysis conducted thus far: that the company sells a lot of clothes while making the most money from bikes and relatively little revenue from bike accessories/parts.

Lastly, which of our customers gave us the most business as recorded in this database? To analyze this, I join the sales table to the customer table:

In [24]:
### Which customers have had the most sales?
query = """

SELECT TOP 10 C.FirstName, C.LastName, C.CompanyName, C.CustomerID, SUM(S.SubTotal) AS TotalRevenue
  FROM SalesLT.SalesOrderHeader AS S
  LEFT JOIN SalesLT.Customer AS C
    ON S.CustomerID = C.CustomerID
  GROUP BY C.FirstName, C.LastName, C.CompanyName, C.CustomerID
  ORDER BY TotalRevenue DESC;
  
"""
   
query_result(query)

,FirstName,LastName,CompanyName,CustomerID,TotalRevenue
0,Terry,Eminhizer,Action Bicycle Specialists,29736,108561.8317
1,Krishna,Sunkammurali,Metropolitan Bicycle Supply,30050,98278.6910
2,Christopher,Beck,Bulk Discount Store,29546,88812.8625
3,Kevin,Liu,Eastside Department Store,29957,83858.4261
4,Jon,Grande,Riding Cycles,29796,78029.6898
5,Jeffrey,Kurtz,Many Bikes Store,29929,74058.8078
6,Rebecca,Laszlo,Instruments and Parts Company,29932,63980.9884
7,Anthony,Chor,Extreme Riding Supplies,29660,57634.6342
8,Frank,Campbell,Trailblazing Sports,29938,41622.0511
9,Catherine,Abel,Professional Sales and Service,29485,39785.3304


This is an interesting result: while most of our top clients are bicycle shops, our number 3 client is actually a bulk discount store. All our top clients (at least our top 5 clients) are also fairly similar in their total amount of business conducted with us in terms of revenue.

In [25]:
conn.close()

## Conclusion

In this project, I found a number of intriguing, actionable insights from the AdventureWorks database using SQL. I found that this company, far and away, makes the bulk of its money selling actual bikes, as opposed to clothes or bike accessories. I also identified its top 10 clients in terms of sales revenue. These would be valuable insights to this company, both in helping it evaluate its performance and plan for future business strategies.